In [1]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re

In [2]:
sitemaps = pd.read_xml("https://www.hln.be/sitemap-news.xml")
sitemaps.head()

,loc,lastmod,news,image
0,https://www.hln.be/internet/help-ik-heb-een-pu...,2023-08-07T10:00:00+00:00,NaN,NaN
1,https://www.hln.be/diksmuide/op-wandel-met-de-...,2023-08-03T14:42:12+00:00,NaN,NaN
2,https://www.hln.be/gent/op-wandel-naar-de-uitk...,2023-08-03T14:39:38+00:00,NaN,NaN
3,https://www.hln.be/stekene/nieuwe-reeks-namen-...,2023-08-03T14:38:25+00:00,NaN,NaN
4,https://www.hln.be/sport-in-de-buurt/roman-neu...,2023-08-03T14:37:53+00:00,NaN,NaN


In [3]:
urls = sitemaps["loc"]
print(urls)

0       https://www.hln.be/internet/help-ik-heb-een-pu...
1       https://www.hln.be/diksmuide/op-wandel-met-de-...
2       https://www.hln.be/gent/op-wandel-naar-de-uitk...
3       https://www.hln.be/stekene/nieuwe-reeks-namen-...
4       https://www.hln.be/sport-in-de-buurt/roman-neu...
                              ...                        
1270    https://www.hln.be/brussel/vub-lanceert-oproep...
1271    https://www.hln.be/showbizz/queen-zaait-ongeru...
1272    https://www.hln.be/wielrennen/kijk-tweede-plaa...
1273    https://www.hln.be/dendermonde/rudi-80-en-adel...
1274    https://www.hln.be/mechelen/dierenbescherming-...
Name: loc, Length: 1275, dtype: object


In [4]:
def fetch_cookie(url):
    session = requests.Session()
    cookie = session.get(url).cookies
    return cookie


fetch_cookie(
    "https://www.hln.be/buitenland/wereldjongerendagen-in-lissabon-1-miljoen-katholieke-jongeren-verwacht-waaronder-500-vlamingen~abb18734/"
)

<RequestsCookieJar[Cookie(version=0, name='xAuthId', value='7602a544-8be5-4cd4-9ad5-74eb79d93001', port=None, port_specified=False, domain='myprivacy.dpgmedia.be', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1724769880, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'Lax'}, rfc2109=False), Cookie(version=0, name='authId', value='7602a544-8be5-4cd4-9ad5-74eb79d93001', port=None, port_specified=False, domain='myprivacy.dpgmedia.be', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1724769880, discard=False, comment=None, comment_url=None, rest={'SameSite': 'Lax'}, rfc2109=False)]>

In [5]:
url = urls[0]
response = requests.get(url, cookies=fetch_cookie(url)).text
print(response)

<!DOCTYPE html>
<html dir="ltr" lang="nl" class="no-js  layout--mobile-animate-subnav layout--sidebar-mobile">

	<head>
		<meta charset="utf-8"/>
<meta http-equiv="x-ua-compatible" content="ie=edge"/>

<!-- meta data -->
<meta name="description" content="HLN - Het Laatste Nieuws - Volg het nieuws op de nr1 nieuwssite in België, HLN.be brengt je het allerlaatste nieuws 24/24 en 7/7, uit binnen - en buitenland, evenals dichtbij met nieuws uit je eigen regio."/>
<meta name="copyright" content="DPG Media NV"/>
<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/>
<meta name="apple-itunes-app" content="app-id=403025567"/>
<meta name="twitter:site" content="@HLN_BE"/><meta name="twitter:title" content="Help, ik heb een puber! Met deze 5 tips breng je rust in huis"/><meta name="twitter:description" content="HLN - Het Laatste Nieuws - Volg het nieuws op de nr1 nieuwssite in België, HLN.be brengt je het allerlaatste nieuws 24/24 en 7/7, uit binnen - en buitenla

In [6]:
soup = BeautifulSoup(response, "html.parser")
print(soup)

<!DOCTYPE html>

<html class="no-js layout--mobile-animate-subnav layout--sidebar-mobile" dir="ltr" lang="nl">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<!-- meta data -->
<meta content="HLN - Het Laatste Nieuws - Volg het nieuws op de nr1 nieuwssite in België, HLN.be brengt je het allerlaatste nieuws 24/24 en 7/7, uit binnen - en buitenland, evenals dichtbij met nieuws uit je eigen regio." name="description"/>
<meta content="DPG Media NV" name="copyright"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="app-id=403025567" name="apple-itunes-app"/>
<meta content="@HLN_BE" name="twitter:site"/><meta content="Help, ik heb een puber! Met deze 5 tips breng je rust in huis" name="twitter:title"/><meta content="HLN - Het Laatste Nieuws - Volg het nieuws op de nr1 nieuwssite in België, HLN.be brengt je het allerlaatste nieuws 24/24 en 7/7, uit binnen - en buitenland, evenals dichtbij met nieuws 

In [7]:
def find_my_shit(url):
    title = soup.find_all("h1")
    article_title = title[0].text.strip()
    return article_title


print(find_my_shit(url))

Help, ik heb een puber! Met deze 5 tips breng je rust in huis


In [8]:
article = soup.find("p").text
print(article)

Wie een tiener in huis heeft, begeeft zich vaak op een mijnenveld van emoties. Vijf tips voor een betere verbinding met je puber: van schermtijd tot veerkracht en privacy.  


In [9]:
published_time = soup.find("meta", property="article:published_time")
test = published_time["content"]

regex = re.compile(r"\d\d\d\d-\d\d-\d\d")
test2 = regex.search(test)
# test2 = test.search(r"\d\d\d\d-\d\d-\d\d")
print(test)

2023-08-07T12:00:00+02:00
